In [1]:
import numpy as np
import altair as alt
import pandas as pd
import database as db

query = """
SELECT p.year, p.day, p.star, p.time as personal, s.time as global, p.position as position
FROM personal p
JOIN scores s
ON p.year = s.year AND p.day = s.day AND p.star = s.star
WHERE s.position = 100
"""

conn = db.open_db('aoc.db')
df = db.do_df(conn, query)
df['personal'] /= 60
df['global'] /= 60
df['relative_time'] = df['personal'] / df['global']
print('personal stats imported from the following ', df.value_counts('year'))

d:\Programmeren\adventofcode\aoc_stats aoc.db
personal stats imported from the following  year
2020    50
2021    50
2022    50
2023    50
2024    50
2017    46
dtype: int64


In [2]:
df

,year,day,star,personal,global,position,relative_time
0,2017,3,2,52.000000,23.316667,NaN,2.230164
1,2017,3,1,15.000000,8.483333,NaN,1.768173
2,2017,4,2,20.000000,3.666667,NaN,5.454545
3,2017,4,1,2.000000,1.883333,NaN,1.061947
4,2017,5,2,22.000000,4.766667,NaN,4.615385
...,...,...,...,...,...,...,...
291,2024,23,1,10.616667,1.766667,1141.0,6.009434
292,2024,24,2,206.350000,61.216667,1932.0,3.370814
293,2024,24,1,24.766667,5.316667,1989.0,4.658307
294,2024,25,2,9.400000,4.566667,524.0,2.058394


In [3]:
selection = alt.selection_multi(fields=['year'])

color = alt.condition(selection,
                      alt.Color('year:N', legend=None),
                      alt.value('lightgray'))

chart = alt.Chart(df).mark_point().encode(
    x=alt.X('personal:Q',scale=alt.Scale(type='log', domain=(1, 400))),
    y=alt.Y('global:Q',scale=alt.Scale(type='log', domain=(1,170))),
    color = color,
    tooltip=['year','day','star']
).properties(
    title='Personal time vs global time',
    width=600,
    height=600)


legend = alt.Chart(df).mark_point().encode(
    y=alt.Y('year:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection
)

chart | legend

c:\Program Files\Python311\Lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [4]:
dom = 'log'
chart = alt.Chart(df).mark_point().encode(
    x=alt.X('personal:Q',scale=alt.Scale(type=dom, domain=(1, 400))),
    y=alt.Y('relative_time:Q',scale=alt.Scale(type=dom, domain=(1,30))),
    column = 'year:N',
    row = 'star:O',
    tooltip=['year','day','star']
)
chart 

alt.Chart(...)

In [5]:
# the regression didn't really have a good fit
b  = chart.transform_regression('personal', 'global', params=True, groupby=['year'], method='linear').mark_line()
import altair_transform
print('regressions for all the years, r2 close to 1 is good')
print(altair_transform.extract_data(b))

ModuleNotFoundError: No module named 'altair_transform'

In [7]:
# df.groupby('Category').agg({'Item':'size','shop1':['sum','mean','std'],'shop2':['sum','mean','std'],'shop3':['sum','mean','std']})
stats = ['median', 'mean', 'max', 'min', 'sum']
df2 = df.groupby('year').agg({'global':stats, 'personal':stats, 'relative_time':stats, 'position':stats }).T.reset_index()
df2 = df2.fillna(-1)
for i in range(2,df2.shape[1]):
    df2.iloc[:,i] = df2.iloc[:,i].astype(int)
df2

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_10348\3772342211.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df2.iloc[:,i] = df2.iloc[:,i].astype(int)


year,level_0,level_1,2017,2020,2021,2022,2023,2024
0,global,median,11,8,9,10,10,4
1,global,mean,15,11,16,15,15,9
2,global,max,54,73,76,74,79,61
3,global,min,1,2,1,1,1,1
4,global,sum,702,563,846,794,781,486
5,personal,median,29,22,20,28,31,18
6,personal,mean,40,57,57,47,67,42
7,personal,max,193,402,360,366,715,430
8,personal,min,2,4,1,2,3,4
9,personal,sum,1880,2870,2868,2352,3380,2132


In [9]:
stats = ['median', 'mean', 'max', 'min', 'sum']
# only second star
df2 = df[df['star']==2].groupby('year').agg({'global':stats, 'personal':stats, 'relative_time':stats, 'position':stats }).T.reset_index()
df2 = df2.fillna(-1)
for i in range(2,df2.shape[1]):
    df2.iloc[:,i] = df2.iloc[:,i].astype(int)
df2

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_10348\2860224505.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df2.iloc[:,i] = df2.iloc[:,i].astype(int)


year,level_0,level_1,2017,2020,2021,2022,2023,2024
0,global,median,20,13,12,13,16,7
1,global,mean,20,15,21,20,22,14
2,global,max,54,73,76,74,79,61
3,global,min,3,4,2,2,5,2
4,global,sum,464,394,534,508,564,369
5,personal,median,41,44,31,36,58,28
6,personal,mean,51,76,68,64,110,60
7,personal,max,193,402,360,366,715,430
8,personal,min,13,8,4,3,5,5
9,personal,sum,1174,1911,1720,1611,2761,1512
